In [2]:
from src.agents.newsletter_base import NewsletterAdal
from lightrag.components.model_client import OpenAIClient

newsletter_agent = NewsletterAdal(
    personality={"mood": "optimistic", "risk_tolerance": 0.5},
    seniority=OpenAIClient(),
    seniority_args= {
        "model": "gpt-4o-mini",
        "temperature": 0.9,
    }
)

In [3]:
from src.data_ingestion.db.postgres_db import PostgresConfig
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas as pd
import datetime 

load_dotenv()

db_uri = PostgresConfig(
    host=os.environ["SB_DDBB_HOST"],
    port=os.environ["SB_DDBB_PORT"],
    database=os.environ["SB_DDBB_DATABASE"],
    user=os.environ["SB_DDBB_USER"],
    password=os.environ["SB_DDBB_PWD"]
).get_connection_string()

engine = create_engine(db_uri)

df_current_portfolio = pd.read_sql("SELECT * FROM portfolio WHERE created_at = (SELECT max(created_at) FROM portfolio)", engine)

In [ ]:
df_current_portfolio = pd.read_sql("SELECT * FROM vecs.sp500_news WHERE created_at = (SELECT max(created_at) FROM vecs.sp500_news)", engine)

In [ ]:
newsletter_agent.create_newsletter(
    context=f"""
    Last week the porfolio has been created, so we have included the following assets, in the form (asset, weight):
    {dict(zip(df_current_portfolio["stock_name"], df_current_portfolio["weight"]))}

    The main news of the week are:
    - The Fed has announced a new round of stimulus
    - The ECB has announced a new round of stimulus
    - The Bank of England has announced a new act

    """
)